In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Loan Approval Prediction: EDA and Model Training"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Part 1: Data Loading & Exploratory Data Analysis (EDA)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import joblib\n",
    "\n",
    "# Load the dataset - Make sure to place your dataset in a 'data' folder at the project root\n",
    "# The notebook is in 'notebooks', so we go up one level ('../') and then into 'data'\n",
    "df = pd.read_csv('../data/loan_prediction_dataset.csv') # Make sure this filename is correct"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Initial Inspection"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"First 5 rows of the dataset:\")\n",
    "print(df.head())\n",
    "\n",
    "print(\"\\nDataset Information:\")\n",
    "df.info()\n",
    "\n",
    "print(\"\\nDescriptive Statistics:\")\n",
    "print(df.describe())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\nMissing values per column:\")\n",
    "print(df.isnull().sum())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Data Visualization"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(6, 5))\n",
    "sns.countplot(x='Loan_Status', data=df, palette='viridis')\n",
    "plt.title('Distribution of Loan Approval Status')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Explore relationships between features and the target\n",
    "plt.figure(figsize=(8, 6))\n",
    "sns.countplot(x='Education', hue='Loan_Status', data=df, palette='pastel')\n",
    "plt.title('Education vs. Loan Status')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(10, 6))\n",
    "sns.histplot(df['ApplicantIncome'], bins=30, kde=True, color='skyblue')\n",
    "plt.title('Applicant Income Distribution')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Part 2: Data Preprocessing & Feature Engineering"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Handle Missing Values"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# For numerical columns, use the median for robustness to outliers\n",
    "df['LoanAmount'] = df['LoanAmount'].fillna(df['LoanAmount'].median())\n",
    "df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].median())\n",
    "df['Credit_History'] = df['Credit_History'].fillna(df['Credit_History'].median())\n",
    "\n",
    "# For categorical columns, use the mode (most frequent value)\n",
    "df['Gender'] = df['Gender'].fillna(df['Gender'].mode()[0])\n",
    "df['Married'] = df['Married'].fillna(df['Married'].mode()[0])\n",
    "df['Dependents'] = df['Dependents'].fillna(df['Dependents'].mode()[0])\n",
    "df['Self_Employed'] = df['Self_Employed'].fillna(df['Self_Employed'].mode()[0])\n",
    "\n",
    "# Verify that all missing values are handled\n",
    "print(\"Missing values after handling:\")\n",
    "print(df.isnull().sum())"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Encode Categorical Variables"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Convert target variable first\n",
    "df['Loan_Status'] = df['Loan_Status'].map({'Y': 1, 'N': 0})\n",
    "\n",
    "# Use one-hot encoding for other categorical features\n",
    "df_encoded = pd.get_dummies(df, columns=['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area'], drop_first=True)\n",
    "\n",
    "print(\"Data shape after encoding:\", df_encoded.shape)\n",
    "df_encoded.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Feature Scaling"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.preprocessing import StandardScaler\n",
    "\n",
    "# Drop Loan_ID as it's not a predictive feature\n",
    "df_encoded = df_encoded.drop('Loan_ID', axis=1)\n",
    "\n",
    "X = df_encoded.drop('Loan_Status', axis=1)\n",
    "y = df_encoded['Loan_Status']\n",
    "\n",
    "scaler = StandardScaler()\n",
    "X_scaled = scaler.fit_transform(X)\n",
    "X = pd.DataFrame(X_scaled, columns=X.columns)\n",
    "\n",
    "X.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Part 3: Model Training & Evaluation"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Split the Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.model_selection import train_test_split\n",
    "\n",
    "# stratify=y ensures the proportion of 'Y' and 'N' is the same in both the train and test sets.\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#### Train and Evaluate Models"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.linear_model import LogisticRegression\n",
    "from sklearn.ensemble import RandomForestClassifier\n",
    "from xgboost import XGBClassifier\n",
    "from sklearn.metrics import accuracy_score, classification_report, confusion_matrix\n",
    "\n",
    "# --- Logistic Regression ---\n",
    "print(\"--- Logistic Regression ---\")\n",
    "lr = LogisticRegression(random_state=42)\n",
    "lr.fit(X_train, y_train)\n",
    "lr_preds = lr.predict(X_test)\n",
    "print(\"Accuracy:\", accuracy_score(y_test, lr_preds))\n",
    "print(classification_report(y_test, lr_preds))\n",
    "\n",
    "# --- Random Forest ---\n",
    "print(\"\\n--- Random Forest ---\")\n",
    "rf = RandomForestClassifier(n_estimators=100, random_state=42)\n",
    "rf.fit(X_train, y_train)\n",
    "rf_preds = rf.predict(X_test)\n",
    "print(\"Accuracy:\", accuracy_score(y_test, rf_preds))\n",
    "print(classification_report(y_test, rf_preds))\n",
    "\n",
    "# --- XGBoost ---\n",
    "print(\"\\n--- XGBoost ---\")\n",
    "xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)\n",
    "xgb.fit(X_train, y_train)\n",
    "xgb_preds = xgb.predict(X_test)\n",
    "print(\"Accuracy:\", accuracy_score(y_test, xgb_preds))\n",
    "print(classification_report(y_test, xgb_preds))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Part 4: Model Selection & Saving"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Based on the results, Logistic Regression and Random Forest perform very well.\n",
    "# Let's choose Random Forest as it's often more robust.\n",
    "\n",
    "# Ensure the 'app' directory exists\n",
    "import os\n",
    "if not os.path.exists('../app'):\n",
    "    os.makedirs('../app')\n",
    "\n",
    "# Save the model, scaler, and column order to the 'app' directory\n",
    "joblib.dump(rf, '../app/random_forest_model.pkl')\n",
    "joblib.dump(scaler, '../app/scaler.pkl')\n",
    "joblib.dump(X.columns, '../app/model_columns.pkl')\n",
    "\n",
    "print(\"Model, scaler, and columns saved successfully in the 'app' directory.\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.7"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
